In [ ]:
import pandas as pd
import re
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import KNNImputer
from sklearn import preprocessing
import random as rand
import numpy as np
import math

In [ ]:
data = pd.read_csv('games_clean.csv')
labels_num = ['Presence','Memory','ReleaseDate','OriginalCost','DiscountedCost','Achievements','Storage',
                'RatingsBreakdown-Recommended','RatingsBreakdown-Meh','RatingsBreakdown-Exceptional','RatingsBreakdown-Skip',
              'Metacritic']

In [ ]:
#le = preprocessing.LabelEncoder()
#data.loc[data['Metacritic'].notna(),'Metacritic'] = le.fit_transform(data[data['Metacritic'].notna()]['Metacritic'])
dataNum = data[labels_num]
dataNum

In [ ]:
pd.get_dummies(pd.Series(list('bcaa')))

In [ ]:
new_labels_num = ['Presence','Memory','ReleaseDate','OriginalCost','DiscountedCost','Achievements','Storage',
                'RatingsBreakdown-Recommended','RatingsBreakdown-Meh','RatingsBreakdown-Exceptional','RatingsBreakdown-Skip',
              'Metacritic_Bad', 'Metacritic_Good', 'Metacritic_Verygood', 'Metacritic_Verybad']

dataDummy = pd.get_dummies(dataNum, columns=["Metacritic"])
dataDummy['Metacritic_Verybad'] = [0] * len(dataDummy) # Ninguno tiene 3
dataDummy = dataDummy.rename(columns={"Metacritic_Very good": "Metacritic_Verygood"})
dataDummy

In [ ]:
dataDummy['Metacritic_Bad'] = dataDummy['Metacritic_Bad'].astype(int)
dataDummy['Metacritic_Good'] = dataDummy['Metacritic_Good'].astype(int)
dataDummy['Metacritic_Verygood'] = dataDummy['Metacritic_Verygood'].astype(int)
dataDummy['Metacritic_Verybad'] = dataDummy['Metacritic_Verybad'].astype(int)

dataDummy['Metacritic_Bad'].dtype
dataDummy

In [ ]:
m0 = dataDummy.Metacritic_Bad == 0
m1 = dataDummy.Metacritic_Good == 0
m2 = dataDummy.Metacritic_Verygood == 0
m3 = dataDummy.Metacritic_Verybad == 0
dataDummy.loc[m0 & m1 & m2 & m3, "Metacritic_Bad"] = np.nan
dataDummy.loc[m0 & m1 & m2 & m3, "Metacritic_Good"] = np.nan
dataDummy.loc[m0 & m1 & m2 & m3, "Metacritic_Verygood"] = np.nan
dataDummy.loc[m0 & m1 & m2 & m3, "Metacritic_Verybad"] = np.nan

dataDummy[dataDummy['Metacritic_Bad'].notna(),'Metacritic_Bad'] = (dataDummy[dataDummy['Metacritic_Bad'].notna()]['Metacritic_Bad']).astype(int)
dataDummy[dataDummy.Metacritic_Good.notna(),'Metacritic_Good'] = dataDummy[dataDummy.Metacritic_Good.notna()].astype(int)
dataDummy[dataDummy.Metacritic_Verygood.notna(),'Metacritic_Verygood'] = dataDummy[dataDummy.Metacritic_Verygood.notna()].astype(int)
dataDummy[dataDummy.Metacritic_Verybad.notna(),'Metacritic_Verybad'] = dataDummy[dataDummy.Metacritic_Verybad.notna()].astype(int)



#dataDummy[['Metacritic_Bad','Metacritic_Good','Metacritic_Verygood','Metacritic_Verybad']].convert_dtypes()
#pd.to_numeric(dataDummy['Metacritic_Bad'], downcast='signed')

dataDummy

In [ ]:
scaler = MinMaxScaler()
scaler.fit(dataDummy)
dataDummy = pd.DataFrame(scaler.transform(dataDummy), columns = dataDummy.columns)
dataDummy.head()

In [ ]:
imputer = KNNImputer(n_neighbors=5,weights='distance')
#dataNumImp = imputer.fit_transform(dataNum)
#dfNum = pd.DataFrame(dataNumImp, columns=labels_num)
dataNumImp = imputer.fit_transform(dataDummy)
dfNum = pd.DataFrame(dataNumImp, columns=new_labels_num)

In [ ]:
for i in range(len(dfNum['ReleaseDate'])):
    dfNum['ReleaseDate'][i] = math.floor(dfNum['ReleaseDate'][i])
    
for i in range(len(dfNum['OriginalCost'])):
    dfNum['OriginalCost'][i] = round(dfNum['OriginalCost'][i], 2)

cols_to_round = ['Achievements', 'RatingsBreakdown-Recommended', 'RatingsBreakdown-Meh', 
                'RatingsBreakdown-Exceptional', 'RatingsBreakdown-Skip']
for colunm in cols_to_round:
    for i in range(len(dfNum[colunm])):
        dfNum[colunm][i] = round(dfNum[colunm][i])

In [ ]:
#dataDummy = pd.DataFrame(scaler.inverse_transform(dataDummy), columns = dataDummy.columns)
#dataDummy.head()

In [ ]:
for name in dfNum.columns:
    lis = dfNum[name].unique()
    print('\033[1m'+name+'\033[0;0m')
    print(lis)
    print('Number of na: ' +str(dfNum[name].isna().sum()))

In [ ]:
dfNum.head()

In [ ]:
dfNum[dfNum.Metacritic_1 >= 0.5].head()

In [ ]:
dfNum[dfNum.Metacritic_2 >= 0.5].head()

In [ ]:
X = dfNum.drop(['Metacritic'],axis=1)         # Data
y = dfNum['Metacritic'].astype(int)           # Labels

print(X.shape)
print(y.shape)

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
import sklearn.neighbors as nb

cv = StratifiedKFold(n_splits=10) 

cv_scores = cross_val_score(nb.KNeighborsClassifier(), X=X,y=y, cv=cv, scoring='accuracy')  
np.mean(cv_scores)

In [ ]:
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(dfNum,labels_num)

In [ ]:
dataNum.hist(figsize=(20,20))

In [ ]:
dataNumImp.hist(figsize=(20,20))